In [ ]:
from context import kolihack
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from kolihack.io import load_content_file

In [ ]:
df = load_content_file()

In [ ]:
df.describe()

In [ ]:
df